<a href="https://colab.research.google.com/github/HausReport/MightyLogic/blob/master/Notebooks/Turf_War_Demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Hello!

If you're seeing this on GitHub, you can open the document on Google by clicking the "Open in Collab" button above.  Once this document is open in Google, you can see the interactive content by selecting `Runtime` on the menubar and then `Run All.`


# Set up environment


In [1]:
import sys
# need newer plotly/plotly_express than collab provides?
#!{sys.executable} -m pip install --force-reinstall --upgrade "plotly"
#!{sys.executable} -m pip install --force-reinstall --upgrade "plotly_express"

#
# Moving some of the guts to GitHub and refactoring 
#
!{sys.executable} -m pip install --upgrade --force-reinstall "git+https://github.com/HausReport/MightyLogic.git#egg=MightyLogic"

#
# Imports
#
from random import randrange
from scipy.signal import convolve2d
from enum import Enum, auto
from pprint import pprint
import plotly.figure_factory as ff
import datetime

import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import math

#import sklearn
from sklearn.preprocessing import PowerTransformer, QuantileTransformer
import numpy as np
from sklearn.preprocessing import StandardScaler, scale

from MightyLogic.Currency import Currency as Currency
from MightyLogic.Values import Values as Values
from MightyLogic.TurfWar.Tiles.BuildingFactory import *
from MightyLogic.TurfWar.TurfWarMap import TurfWarMap
from MightyLogic.TurfWar.Tiles.TileList import TileList
#Values.GOLD

  Cloning https://github.com/HausReport/MightyLogic.git to /tmp/pip-install-ydzei3n9/mightylogic_6e2f10c85d6640cda09a121a5567497f
  Running command git clone -q https://github.com/HausReport/MightyLogic.git /tmp/pip-install-ydzei3n9/mightylogic_6e2f10c85d6640cda09a121a5567497f
     |████████████████████████████████| 280 kB 36.1 MB/s 
     |████████████████████████████████| 40 kB 6.0 MB/s 
     |████████████████████████████████| 53 kB 2.3 MB/s 
     |████████████████████████████████| 97 kB 7.0 MB/s 
     |████████████████████████████████| 67 kB 6.2 MB/s 
  Created wheel for MightyLogic: filename=MightyLogic-0.2-py3-none-any.whl size=23589 sha256=36ae95ad2eb4a411c37bd85fabff6740405da785eab1a7148004e82707854d3c
  Stored in directory: /tmp/pip-ephem-wheel-cache-6xwgny_5/wheels/84/7a/55/d74e49b550ce0fddb2379db726853920650df706dedea8daae
Successfully built MightyLogic
  Attempting uninstall: zipp
    Found existing installation: zipp 3.5.0
    Uninstalling zipp-3.5.0:
      Successfully unin

# Turf War Demo

In [2]:

#
# A Turf War Map
#

map2 =TurfWarMap()
map2.setGuild("Nightterror")
map2.setStartDate(year=2021, month=7, day=26)


# Use convenience functions to build value array
map2.addTile('A',1, mountains(1600,1000))
map2.addTile('A',2, forest(1600,0,1000))
map2.addTile('A',3, plains(1600, 1000))
map2.addTile('A',4, mountains(1600,1000))
map2.addTile('A',5, forest(1600,0,1000))
map2.addTile('A',6, plains(1600,1000))

map2.addTile('B',1, plains(1600,1000))
map2.addTile('B',2, witch_hut(1700,300,3500))
map2.addTile('B',3, forest(2500,0,2500))
map2.addTile('B',4, altar(300,400,4250))
map2.addTile('B',5, mountains(2500,2500))
map2.addTile('B',6, forest(1600,0,1000))

map2.addTile('C',1, forest(1600,0,1000))
map2.addTile('C',2, plains(2500,2500))
map2.addTile('C',3, stronghold(300,400,4250))
map2.addTile('C',4, forest(2500,0,2500))
map2.addTile('C',5, hells_gate(300,400,4250))
map2.addTile('C',6, mountains(1600,1000))

map2.addTile('D',1, gold_mine(45000,250,2500))
map2.addTile('D',2, forest(2500,0,2500))
map2.addTile('D',3, plains(2500,2500))
map2.addTile('D',4, mountains(2500,2500))
map2.addTile('D',5, forest(2500,0,2500))
map2.addTile('D',6, factory(900, 250, 2500))

map2.addTile('E',1, plains(1600,1000))
map2.addTile('E',2, mountains(1600,1000))
map2.addTile('E',3, forest(1600,0,1000))
map2.addTile('E',4, plains(1600,1000))
map2.addTile('E',5, mountains(1600,1000))
map2.addTile('E',6, forest(1600,0,1000))


# Payout array
image = map2.getValues()
res = map2.stagingScores()
#pprint(image)

In [3]:
#
# An approach that shows well-labeled heatmaps.
#

def get_heatmap(arr, title, round=True):
  flip = np.flip( np.array(arr), axis=0)  # use numpy to flip array vertically
  z = flip
  if round:
    z = z.round()

  x = ['1', '2', '3', '4','5','6']
  y = ['E','D','C','B','A']

  #z_text = ["%.0f" % number for number in z] - doesn't work

  if round:
    z_text = flip.round( decimals=-1)
  else:
    z_text = flip.round( decimals=3)
  # Possible to build up z_text to make it more friendly
  # z_text = [['Win', 'Lose', 'Win'],
  #          ['Lose', 'Lose', 'Win'],
  #          ['Win', 'Win', 'Lose']]

  fig = ff.create_annotated_heatmap(z, x=x, y=y, annotation_text=z_text, colorscale='Viridis')
  fig.update_layout(title=title, autosize=False,
                  width=500, height=500,
                  margin=dict(l=65, r=50, b=65, t=90))
  return fig #fig.show()



In [4]:
#
# The payout array can be accessed as a Pandas dataframe.
#
payout = map2.getDataFrame()
payout

,1,2,3,4,5,6
A,15001.6,15001.6,15001.6,15001.6,15001.6,15001.6
B,15001.6,57017.0,37502.5,69780.0,37502.5,15001.6
C,15001.6,37502.5,69780.0,37502.5,69780.0,15001.6
D,68250.0,37502.5,37502.5,37502.5,37502.5,41295.0
E,15001.6,15001.6,15001.6,15001.6,15001.6,15001.6


In [5]:
#
# The staging array can be accessed as a Pandas dataframe.
#
staging =map2.getStrategicDataFrame()
staging

,1,2,3,4,5,6
A,30003.2,87020.2,67505.7,99783.2,67505.7,30003.2
B,87020.2,0.0,211578.6,0.0,169563.2,67505.7
C,120754.1,179301.1,0.0,246842.5,0.0,126076.6
D,0.0,158256.6,159786.6,127509.1,163579.1,0.0
E,83251.6,67505.7,67505.7,67505.7,67505.7,56296.6


In [6]:
#
# Heatmaps show the best tiles for payout round and non-payout rounds.
#


fig1 = get_heatmap(image,"Payout Values for 3rd Rounds")
fig2 = get_heatmap(res,"Strategic Values for Non-Payout Rounds")

fig1.show()
fig2.show()

In [7]:
#
# Shows payout shares for holding a tile on round 3n 
#

gm = map2.getTile('B',6) #b6 # c5 # gold_mine(90000,500,5000)
print(gm.payouts())


Payouts for B:6 Forest
Place               Reward
First               112 common souls, 70 contribution
2nd                 96 common souls, 60 contribution
3rd                 80 common souls, 50 contribution
4-6                 64 common souls, 40 contribution
7-11                48 common souls, 30 contribution
12-25               32 common souls, 20 contribution
26-44               16 common souls, 10 contribution
0 guild influence


In [8]:
#
# List of buildings easy for players to read.
#
map2.printBuildingList()

Building          Points      For 1st Place                      Guild Gets
B-4: Altar         69,780      21 epic souls, 298 contribution,  400 influence.
C-3: Stronghold    69,780      21 epic souls, 298 contribution,  400 influence.
C-5: Hell's Gate   69,780      21 epic souls, 298 contribution,  400 influence.
D-1: Gold Mine     68,250         3,150 gold, 175 contribution,  250 influence.
B-2: Witch Hut     57,017     119 rare souls, 245 contribution,  300 influence.
D-6: Factory       41,295       63 soul dust, 175 contribution,  250 influence.


In [9]:
#
# Text report that highlights building returns 
#

map2.printReport()

Turf War Map Report
Guild: Nightterror
For the Week of: 26 July 2021
Total Map Value: 915,948 common souls

Building          Points      For 1st Place                      Guild Gets
B-4: Altar         69,780      21 epic souls, 298 contribution,  400 influence.
C-3: Stronghold    69,780      21 epic souls, 298 contribution,  400 influence.
C-5: Hell's Gate   69,780      21 epic souls, 298 contribution,  400 influence.
D-1: Gold Mine     68,250         3,150 gold, 175 contribution,  250 influence.
B-2: Witch Hut     57,017     119 rare souls, 245 contribution,  300 influence.
D-6: Factory       41,295       63 soul dust, 175 contribution,  250 influence.


In [10]:
#
# Evaluate neighbors of a given tile
#
map2.printNeighborList('B',2)

Neighbors of tile B:2
Building          Points      For 1st Place                      Guild Gets
B-1: Plains        15,002    112 common souls, 70 contribution,    0 influence.
B-3: Forest        37,502   175 common souls, 175 contribution,    0 influence.
A-2: Forest        15,002    112 common souls, 70 contribution,    0 influence.
C-2: Plains        37,502   175 common souls, 175 contribution,    0 influence.


# Strategy & Payout Arrays

In [11]:
#
# High values are desirable payout locations, low are less desirable
#

#pd.DataFrame(map2.getScaledPayout())

In [12]:
#
# High values are desirable staging locations, low are less desirable
#

#pd.DataFrame(map2.getScaledStaging())

In [13]:
#
# Lowest values are least desirable staging locations, higher are better staging locations, highest are desirable payout locations
#
adjusted_move = map2.getAdjustedMoves()
pd.DataFrame(adjusted_move)

,0,1,2,3,4,5
0,0.000000,0.285825,0.187999,0.349805,0.187999,0.000000
1,0.285825,2.032991,1.018889,2.696240,1.018889,0.187999
2,0.454932,1.018889,2.696240,1.087009,2.696240,0.481613
3,2.616731,1.018889,1.018889,1.018889,1.018889,1.215972
4,0.266933,0.187999,0.187999,0.187999,0.187999,0.131808


In [14]:
moves = adjusted_move
get_heatmap(moves,"Movement Array", round=False).show()

In [15]:
#
# 3D Presentation of the movement surface
#
flip = np.flip( moves, axis=1) 
fig = go.Figure(data=[go.Surface(z=flip)])
fig.update_layout(title='Movement Array in 3D', autosize=False,
                  width=500, height=500,
                  margin=dict(l=65, r=50, b=65, t=90))
fig.show()

In [16]:
retx, rety = map2.getRecommendedMovesArrays()

In [17]:
def getTileTrace():

  # Generate tile name array
  cnames = []
  rtl = ['E','D','C','B','A']
  ctli = [1,2,3,4,5,6]
  for c in ctli:
    for r in rtl:
      cnames.append(f"{r}{c}")
      
  return go.Scatter(x=[0,0,0,0,0,1,1,1,1,1,2,2,2,2,2,3,3,3,3,3,4,4,4,4,4,5,5,5,5,5], y=[0,1,2,3,4,0,1,2,3,4,0,1,2,3,4,0,1,2,3,4,0,1,2,3,4,0,1,2,3,4],
                          mode='markers+text',
                          textposition="top right",
                          text = cnames,
                          textfont=dict(
                            family="sans serif",
                            size=18,
                            color="red"
                          ),
                          marker_size=14,
                          name='Tiles')
  
def getContourTrace(data):
  z = np.flip( data, axis=0)
  return(go.Contour(
        z=z,
        x=[0,1,2,3,4,5], # horizontal axis
        y=[0,1,2,3,4],   # vertical axis
        colorscale="delta"
        #showlegend=False
        #color_continuous_scale=px.colors.sequential.Viridis
    ))

def squaresTrace():
  pass
  #fig.add_shape(type="rect",
  #  xref="x", yref="y",
  #  x0=-0.5, y0=-0.5,
  #  x1=0.5, y1=0.5,
  #  line=dict(
  #      color="RoyalBlue",
  #      width=3,
  #  ),
  #  #fillcolor="LightSkyBlue",
  # #)

In [18]:
#
# Show staging values as a contour plot
#
fig = go.Figure()
fig.add_trace( getContourTrace(map2.getStrategicDataFrame().to_numpy()) )
fig.add_trace( getTileTrace())
fig.update_layout(title='Strategic Value of Staging Tiles')
fig.update_xaxes(visible=False)
fig.update_yaxes(visible=False)
fig.update_layout(dict(paper_bgcolor='White',plot_bgcolor='White', showlegend=False))
fig.show()

In [19]:
  #fig = ff.create_streamline(x, y, u,v, arrow_scale=.1, name='streamline')
  x,y = np.meshgrid(np.arange(0, 6, 1), np.arange(0, 5, 1))
  fig = ff.create_quiver(x,y,                       #positions 
                         retx, rety,                #vectors
                         scale=1,                   #vector scale
                         name="Recommended Move",   #legend title
                         line=dict(width=2,
                                color='red'),
                        )

  # add dots at integers
  fig.add_trace( getTileTrace())

  # add contour plot   
  z = payout.to_numpy()
  fig.add_trace( getContourTrace(z) )
  
  # cosmetics
  #  - axes are confusing & extraneous
  fig.update_xaxes(visible=False)
  fig.update_yaxes(visible=False)

  #  - KISS   
  showLegend = False #dots & arrows  # 3 legends are colliding
  fig.update_layout(dict(paper_bgcolor='White',plot_bgcolor='White', showlegend=showLegend))

  #fig.update_layout dict(show_legend=False))
  #fig.update_layout(coloraxis_showscale=False)

  # move colorbar
  #fig.update_layout(coloraxis_colorbar=dict(yanchor="top", y=1, x=0,
  #                                        ticks="outside",
  #                                        ticksuffix=" bills"))

  # - graph title
  fig.update_layout(title='Relative Payout and Recommended Moves')
  fig.show()

# Multi-Tile Targets

In [20]:
import re
s = 'a1a5c6e3'
res = re.findall('(\d+|[A-Za-z]+)', s)
print(res)

def parse_tile(inp):
  pass

['a', '1', 'a', '5', 'c', '6', 'e', '3']


In [21]:
image = map2.getValues()


#pprint(image)
filter_kernel =  [[0, 0, 0],
                  [0, 1, 1],
                  [0, 1, 0]]

def getComboValues(filter_kernel):
  filter_kernel = np.flipud(np.fliplr(filter_kernel))
  res = convolve2d(image,
                 filter_kernel,
                 mode='same',
                 boundary='fill',
                 fillvalue=0)
  return res

#pprint(res)
#fig1 = get_heatmap(image,"Payout Values for 3rd Rounds")
#fig1.show()

filter_kernel0 =  [[0, 0, 0],
                  [0, 1, 1],
                  [0, 1, 0]]
drcombos = getComboValues(filter_kernel)

filter_kernel1 =  [[0, 1, 0],
                  [0, 1, 1],
                  [0, 0, 0]]
urcombos = getComboValues(filter_kernel)

filter_kernel2 =  [[0, 0, 0],
                  [1, 1, 0],
                  [0, 1, 0]]
dlcombos = getComboValues(filter_kernel)

filter_kernel3 =  [[0, 1, 0],
                  [1, 1, 0],
                  [0, 0, 0]]
ulcombos = getComboValues(filter_kernel)

filter_kernel4 =  [[0, 0, 0],
                  [1, 1, 1],
                  [0, 0, 0]]
hcombos = getComboValues(filter_kernel)

filter_kernel5 =  [[0, 1, 0],
                  [0, 1, 0],
                  [0, 1, 0]]
vcombos = getComboValues(filter_kernel)

filter_kernels = [filter_kernel0,filter_kernel1,filter_kernel2,filter_kernel3,filter_kernel4,filter_kernel5]
#get_heatmap(drcombos,"down-right combos  ", round=False)
get_heatmap(urcombos,"up-right combos  ", round=False)
get_heatmap(hcombos,"horiz combos  ", round=False)
get_heatmap(vcombos,"vert combos  ", round=False)

#drlcombos.show()

In [22]:

# given row in [0-4] and col in [0-5], return array of valid tile locations
# matching the given convolution matrix

def getFilterTileLocations(r, c, filter_kernel):
  row_dict = {0:'A', 1:'B', 2: 'C', 3:'D', 4: 'E'}
  row=row_dict[r]
  j=c+1

  uprow = map2.upRow(row)
  leftcol = map2.leftCol(j)
  rightcol = map2.rightCol(j)
  dnrow = map2.downRow(row)

  ret = []
  print(uprow)
  # above row
  if uprow is not None:
    if leftcol is not None:
      if filter_kernel[0][0]==1:
        #print(uprow + str(leftcol))
        ret.append([uprow, leftcol])
    if filter_kernel[0][1]==1:
      #print(uprow + str(j))
      ret.append([uprow, j])
    if rightcol is not None:
      if filter_kernel[0][2]==1:
        #print(uprow + str(rightcol))
        ret.append([uprow, rightcol])

  # this row
  if leftcol is not None:
    if filter_kernel[1][0]==1:
      #print(row + str(leftcol))
      ret.append([row, leftcol])
  if filter_kernel[1][1]==1:
    #print(row + str(j))
    ret.append([row, j])
  if rightcol is not None:
    if filter_kernel[1][2]==1:
      #print(row + str(rightcol))
      ret.append([row, rightcol])

  # below row
  if dnrow is not None:
    if leftcol is not None:
      if filter_kernel[2][0]==1:
        #print(dnrow + str(leftcol))
        ret.append([dnrow, leftcol])
    if filter_kernel[2][1]==1:
      #print(dnrow + str(j))
      ret.append([dnrow, j])
    if rightcol is not None:
      if filter_kernel[2][2]==1:
        #print(dnrow + str(rightcol))
        ret.append([dnrow, rightcol])
  return ret

ret = getFilterTileLocations(0, 0, filter_kernel)
pprint(ret)
tarr = map2.getTiles(ret)
tlist = TileList(map2,ret)
#tlist.share(0.07)

None
[['A', 1], ['A', 2], ['B', 1]]


In [23]:
allCombos = []
for filter in filter_kernels:
  for r in [0,1,2,3,4]:
    for c in [0,1,2,3,4,5]:
      allCombos.append(TileList(map2, getFilterTileLocations(r, c, filter_kernel)))

allCombos = set(allCombos)

None
None
None
None
None
None
A
A
A
A
A
A
B
B
B
B
B
B
C
C
C
C
C
C
D
D
D
D
D
D
None
None
None
None
None
None
A
A
A
A
A
A
B
B
B
B
B
B
C
C
C
C
C
C
D
D
D
D
D
D
None
None
None
None
None
None
A
A
A
A
A
A
B
B
B
B
B
B
C
C
C
C
C
C
D
D
D
D
D
D
None
None
None
None
None
None
A
A
A
A
A
A
B
B
B
B
B
B
C
C
C
C
C
C
D
D
D
D
D
D
None
None
None
None
None
None
A
A
A
A
A
A
B
B
B
B
B
B
C
C
C
C
C
C
D
D
D
D
D
D
None
None
None
None
None
None
A
A
A
A
A
A
B
B
B
B
B
B
C
C
C
C
C
C
D
D
D
D
D
D


In [24]:
allCombos = list(allCombos)
allCombos.sort(key=lambda x: x.value, reverse=True)
for combo in allCombos:
  val = combo.getValue()
  print(combo.getTileLine()) #str(val) + " - " + combo.getName())

B:3, B:4, C:3:  177,062  175 common souls, 42 epic souls, 770 contribution,  800 influence.
B:4, B:5, C:4:  144,785  350 common souls, 21 epic souls, 648 contribution,  400 influence.
C:2, C:3, D:2:  144,785  350 common souls, 21 epic souls, 648 contribution,  400 influence.
C:3, C:4, D:3:  144,785  350 common souls, 21 epic souls, 648 contribution,  400 influence.
C:4, C:5, D:4:  144,785  350 common souls, 21 epic souls, 648 contribution,  400 influence.
B:2, B:3, C:2:  132,022  350 common souls, 119 rare souls, 595 contribution,  300 influence.
B:5, B:6, C:5:  122,284  287 common souls, 21 epic souls, 542 contribution,  400 influence.
C:5, C:6, D:5:  122,284  287 common souls, 21 epic souls, 542 contribution,  400 influence.
C:1, C:2, D:1:  120,754  287 common souls, 3,150 gold, 420 contribution,  250 influence.
D:1, D:2, E:1:  120,754  287 common souls, 3,150 gold, 420 contribution,  250 influence.
A:4, A:5, B:4:   99,783  224 common souls, 21 epic souls, 438 contribution,  400 infl

In [25]:

topTen = allCombos[:10]
print("Top 3-Tile Combos on Map")
print("Tiles           Points   Reward for first place")
print("--------------------------------------------------------------------------------------------------------")
for combo in topTen:
  print(combo.getTileLine())

Top 3-Tile Combos on Map
Tiles           Points   Reward for first place
--------------------------------------------------------------------------------------------------------
B:3, B:4, C:3:  177,062  175 common souls, 42 epic souls, 770 contribution,  800 influence.
B:4, B:5, C:4:  144,785  350 common souls, 21 epic souls, 648 contribution,  400 influence.
C:2, C:3, D:2:  144,785  350 common souls, 21 epic souls, 648 contribution,  400 influence.
C:3, C:4, D:3:  144,785  350 common souls, 21 epic souls, 648 contribution,  400 influence.
C:4, C:5, D:4:  144,785  350 common souls, 21 epic souls, 648 contribution,  400 influence.
B:2, B:3, C:2:  132,022  350 common souls, 119 rare souls, 595 contribution,  300 influence.
B:5, B:6, C:5:  122,284  287 common souls, 21 epic souls, 542 contribution,  400 influence.
C:5, C:6, D:5:  122,284  287 common souls, 21 epic souls, 542 contribution,  400 influence.
C:1, C:2, D:1:  120,754  287 common souls, 3,150 gold, 420 contribution,  250 influen

In [26]:
best = TileList(map2, [['C', 3], ['B', 2], ['B', 4], ['C',1]])
print(best.payouts())


Payouts for Stronghold at C:3, Witch Hut at B:2, Altar at B:4, Forest at C:1
Place               Reward
First               112 common souls, 119 rare souls, 42 epic souls, 910 contribution
2nd                 96 common souls, 102 rare souls, 36 epic souls, 780 contribution
3rd                 80 common souls, 85 rare souls, 30 epic souls, 650 contribution
4-6                 64 common souls, 68 rare souls, 24 epic souls, 520 contribution
7-11                48 common souls, 51 rare souls, 18 epic souls, 390 contribution
12-25               32 common souls, 34 rare souls, 12 epic souls, 260 contribution
26-44               16 common souls, 17 rare souls, 6 epic souls, 130 contribution



In [98]:
# round 1
# boost +70 200
# +40 100
# +20 30

# round 2
# +90 800
# +60 400
# +30 150

class PlayerGroup():
  def __init__(self):
    self.players = []

  def append(self,player):
    self.players.append(player)

  def strength(self):
    sum = 0
    for player in self.players:
      sum += player.getTroops()
    return sum

  def sort(self):
    #keyfun= operator.attrgetter("getTroops") # use operator since it's faster than lambda
    self.players.sort(reverse=True) # sort in-place

  def print(self, title="",time=True):
    if time:
      print("Report Generated at " + datetime.datetime.utcnow().strftime("%H:%M GMT on %d %B %Y"))
    self.sort()
    sum = 0
    if title != "":
      print(title)
    for player in self.players:
      player.print()
      sum += player.getTroops()
    print("Team troops: " + str(sum))

  # https://stackoverflow.com/questions/5248954/an-algorithm-to-sort-a-list-of-values-into-n-groups-so-that-the-sum-of-each-grou
  def split(self, nTeams):
    groups = [PlayerGroup() for i in range(nTeams)]
    self.sort()
    awol_cooldown = PlayerGroup()
    #print("Report Generated at " + datetime.datetime.utcnow().strftime("%H:%M GMT on %d %B %Y"))
    #spent = PlayerGroup()
    for x in self.players:
      if x.awol or x.cooldown:
        awol_cooldown.append(x)
      elif x.getTroops()<1:
        awol_cooldown.append(x)
        #spent.append(x)
      else:
        mingroup = groups[0]
        for g in groups:
            if g.strength() < mingroup.strength():
              mingroup = g
        mingroup.append(x)
    groups.append(awol_cooldown)
    #groups.append(spent)
    return groups

class Player():
  def __init__(self, name, troops, elixirBoost=0, spent = 0, cooldown=False, awol=False):
    self.name = name
    self.troops=troops
    self.spent=spent
    self.elixirBoost=elixirBoost
    self.cooldown = cooldown
    self.awol = awol

  def getOrder(self, amt):
    return self.getTroops()/3

  def getNature(self, amt):
    return self.getTroops()/3

  def getChaos(self, amt):
    return self.getTroops()/3

  def getTroops(self):
    if self.cooldown or self.awol:
      return 0
    avail = self.troops - self.spent
    if avail<0:
      avail=0
    return (avail) *((100+self.elixirBoost)/100)

  def print(self):
    tr = self.getTroops()
    str = f"{tr:>8,.0f} {self.name}"
    if self.elixirBoost>0:
      str += f" (elixir boost {self.elixirBoost}%)"
    if self.spent > 0:
      str += f" (spent:{self.spent:,.0f})"
    if self.awol:
      str += f" (AWOL)"
    elif self.cooldown:
      str += f" (Cooldown)"
    elif self.getTroops()<1:
      str += f" (Spent)"
    print(str)


  def __eq__(self, other):
    return self.name == other.name #and self.geT == other.suit

  def __lt__(self, other):
        return self.getTroops() < other.getTroops()

  #def assign(self, tileNum = 0, order=0, chaos=0, nature=0, elixirBoost=0):
  #  ret = [0 for i in range(3)]
  #  ret[tileNum] = (self.getOrder(1)+self.getChaos(1)+ self.getNature(1))*((100+elixirBoost)/100)
  #  return ret

desired = [301911-25910,142158-24297, 70193-12392]
print(desired)

#players = [ Player("Malbad", 56436), Player("BilB", 19894), Player("Stalguard",31788), Player("Eragon", 28602)]

players = PlayerGroup()
total = []
players.append(Player("Rags",64927,70, spent=29942))
players.append(Player("BilBisqqit",59909,70))
players.append(Player("Malbad",56436,70))
players.append(Player("Hashem",56118,70))
players.append(Player("Panda",52514,70))
players.append(Player("chingcheng",50540,70))


players.append(Player("LordOtto",45669, spent=17509))
players.append(Player("Flowbites",41781,cooldown=True))
players.append(Player("Erlaed",40705,70, spent=4063))
players.append(Player("Sente",38083,cooldown=True))

players.append(Player("negro",32510))
players.append(Player("FriarKen",31473))
players.append(Player("Stalguard",31788,70))

players.append(Player("OmniMan",30975))
players.append(Player("ThaBozz",30542))
players.append(Player("Wheeler",29897, spent=9512))
players.append(Player("Vasilich",29125))

players.append(Player("Eragon",28602,70))
players.append(Player("Litch",27547, awol=True))
players.append(Player("iahobo",27255))
players.append(Player("RoMiC",26482))
players.append(Player("Ender",26283, spent=30025))
players.append(Player("PiXeL1K",26017))
players.append(Player("Tech34",25753))
players.append(Player("Hung",25173))
players.append(Player("Iceman",24952, spent=103))
players.append(Player("Seph",19118))
players.append(Player("Beeznutz",18649))
players.append(Player("Ramzinator",14318))
players.append(Player("Holymight",13941))
players.append(Player("Nikki",11759))

players.sort()
players.print()

print("Report Generated at " + datetime.datetime.utcnow().strftime("%H:%M GMT on %d %B %Y"))
four_groups= players.split(4)
i=0
for group in four_groups:
  group.print(title="Team " + str(i), time=False)
  i +=1
#for player in players:
#  r = player.assign(0,1,1,1,0)
#  total.append(r)



[276001, 117861, 57801]
Report Generated at 05:52 GMT on 29 July 2021
 101,845 BilBisqqit (elixir boost 70%)
  95,941 Malbad (elixir boost 70%)
  95,401 Hashem (elixir boost 70%)
  89,274 Panda (elixir boost 70%)
  85,918 chingcheng (elixir boost 70%)
  62,291 Erlaed (elixir boost 70%) (spent:4,063)
  59,474 Rags (elixir boost 70%) (spent:29,942)
  54,040 Stalguard (elixir boost 70%)
  48,623 Eragon (elixir boost 70%)
  32,510 negro
  31,473 FriarKen
  30,975 OmniMan
  30,542 ThaBozz
  29,125 Vasilich
  28,160 LordOtto (spent:17,509)
  27,255 iahobo
  26,482 RoMiC
  26,017 PiXeL1K
  25,753 Tech34
  25,173 Hung
  24,849 Iceman (spent:103)
  20,385 Wheeler (spent:9,512)
  19,118 Seph
  18,649 Beeznutz
  14,318 Ramzinator
  13,941 Holymight
  11,759 Nikki
       0 Flowbites (Cooldown)
       0 Sente (Cooldown)
       0 Litch (AWOL)
       0 Ender (spent:30,025) (Spent)
Team troops: 1129291.7999999998
Report Generated at 05:52 GMT on 29 July 2021
Team 0
 101,845 BilBisqqit (elixir boost 70

'29 July 2021 at 05:46'

# Notes


1.   Annotated heatmap: https://plotly.com/python/annotated-heatmap/
2.   A-Star search on grid in python at https://github.com/BaijayantaRoy/Medium-Article/blob/master/A_Star.ipynb
1. Add payout report for multiple tiles

